# Trip Advisor Hotel Review NLP Project

Coded by Luna McBride.

The purpose for this project is to test Keras for Natural Language Processing. Thus, I will use the keras tokenizer and stuff like that to process instead of Spacy, like I used previously.

In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf #Import tensorflow in order to use Keras
from tensorflow.keras.preprocessing.text import Tokenizer #Add the keras tokenizer for tweet tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add padding to help the Keras Sequencing
import tensorflow.keras.layers as L #Import the layers as L for quicker typing
from tensorflow.keras.optimizers import Adam #Pull the adam optimizer for usage

from tensorflow.keras.losses import SparseCategoricalCrossentropy #Loss function being used
from sklearn.model_selection import train_test_split #Train Test Split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv


In [18]:
hotel = pd.read_csv("../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv") #Load the review data into pandas
hotel.head() #Take a peek at the dataset

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


---

# Check for Nulls

In [19]:
print(hotel.isnull().any()) #Check if there are any null values
print(hotel["Rating"].value_counts()) #Checks the rating values in case there is a weird value
print(hotel.loc[hotel["Review"] == ""]) #Checks for empty review strings

Review    False
Rating    False
dtype: bool
5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64
Empty DataFrame
Columns: [Review, Rating]
Index: []


There are no null values to look at here. The Ratings column also only has values from 1 to 5, so no weird values. There are definitely more positive than negative reviews, so that is something to keep in mind for the model. As for ratings, there are no blank strings. That means there is no cleaning that needs to be done in that regard.

In [20]:
print(hotel["Review"][26]) #Print a random review to show off the structure.

loved, stayed warwick overnight getway enjoy christmas shopping, warwick exceeded expectations, staff wonderful extrememly friendly room clean service lounge wonderful, came contact hotel friendly, women bathroom lever lounge well.. think haunted totally creepy vibe lights anywho, really enjoyed stay going couple days,  


It also seems as though the reviews have all had their stopwords removed, as seen by the review above. There is still some punctuation I may want to remove, so I will still do that.

---

# Review Punctuation Cleaning

Attached punctuation guide for future reference: https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string

In [21]:
punctuations = """!()-![]{};:,+'"\,<>./?@#$%^&*_~Â""" #List of punctuation to remove

#ReviewParse: Takes the stubborn punctuation off the words for a single review
#Input: the review to parse
#Output: the parsed review
def reviewParse(review):
    splitReview = review.split() #Split the review into words
    parsedReview = "".join([word.translate(str.maketrans('', '', punctuations)) + " " for word in splitReview]) #Takes the stubborn punctuation out
    return parsedReview #Returns the parsed review

hotel["CleanReview"] = hotel["Review"].apply(reviewParse) #Parse all the reviews for their punctuation and add it into a new column
hotel.head() #Take a peek at the dataset

,Review,Rating,CleanReview
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...
2,nice rooms not 4* experience hotel monaco seat...,3,nice rooms not 4 experience hotel monaco seatt...
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...
4,"great stay great stay, went seahawk game aweso...",5,great stay great stay went seahawk game awesom...


---

# Tokenizing and Padding

In [25]:
review = hotel["CleanReview"].copy() #Use a copy of the clean reviews

#Print an example sentence to make sure everything is working
print("Example Sentence: ") 
print(review[26])

token = Tokenizer() #Initialize the tokenizer
token.fit_on_texts(review) #Fit the tokenizer to the reviews
texts = token.texts_to_sequences(review) #Convert the reviews into sequences for keras to use

#Print an example sequence to make sure everything is working
print("Into a Sequence: ")
print(texts[26])

texts = pad_sequences(texts, padding='post') #Pad the sequences to make them similar lengths

#Print an example padded sequence to make sure everything is working
print("After Padding: ")
print(texts[26])

Example Sentence: 
loved stayed warwick overnight getway enjoy christmas shopping warwick exceeded expectations staff wonderful extrememly friendly room clean service lounge wonderful came contact hotel friendly women bathroom lever lounge well think haunted totally creepy vibe lights anywho really enjoyed stay going couple days 
Into a Sequence: 
[82, 15, 6277, 1614, 18133, 253, 1159, 180, 6277, 1541, 595, 7, 61, 7039, 30, 2, 20, 16, 268, 61, 184, 1175, 1, 30, 1224, 42, 15674, 268, 9701, 91, 11722, 732, 5378, 3328, 1077, 22270, 26, 123, 8, 83, 151, 78]
After Padding: 
[  82   15 6277 ...    0    0    0]


---

# Label Encoding

In [27]:
#EncodeLabel: encode the labels into 0, 1, and 2, back to the issue of explaining positive and extremely positive to a machine
#Input: the star rating
#Output: 0, 1, or 2 indicating rating positivity/negativity
def encodeLabel(label):
    if label == 5 or label == 4: #If the rating is generally positive
        return 2 #Give the rating a 2 for positive
    if label == 3: #If the rating is generally neutral
        return 1 #Give the rating a 1 for neutral
    return 0 #Give the rating a 0 for negative

labels = ["Negative", "Neutral", "Positive"] #Give our labels a name
hotel["EncodedRating"] = hotel["Rating"].apply(encodeLabel) #Encode the ratings to positivity labels
hotel.head() #Take a peek at the dataset with the new labels

,Review,Rating,CleanReview,EncodedRating
0,nice hotel expensive parking got good deal sta...,4,nice hotel expensive parking got good deal sta...,2
1,ok nothing special charge diamond member hilto...,2,ok nothing special charge diamond member hilto...,0
2,nice rooms not 4* experience hotel monaco seat...,3,nice rooms not 4 experience hotel monaco seatt...,1
3,"unique, great stay, wonderful time hotel monac...",5,unique great stay wonderful time hotel monaco ...,2
4,"great stay great stay, went seahawk game aweso...",5,great stay great stay went seahawk game awesom...,2


---

# Model Training

Sources for future use: https://keras.io/guides/working_with_rnns/ , https://www.kaggle.com/shahraizanwar/covid19-tweets-sentiment-prediction-rnn-85-acc 

In [34]:
#Split the data for training and testing
textTrain, textTest, ratingTrain, ratingTest = train_test_split(texts, hotel["EncodedRating"], test_size = 0.33, random_state = 24)

In [35]:
size = len(token.word_index) + 1 #Set the number of words for the size
ratings = hotel["EncodedRating"].copy() #Get the encoded ratings from the dataframe

tf.keras.backend.clear_session() #Clear any previous model building

epoch = 2 #Number of runs through the data
batchSize = 32 #The number of items in each batch
outputDimensions = 16 #The size of the output
units = 256 #Dimensions of the output space

model = tf.keras.Sequential([ #Start the sequential model, doing one layer after another in a sequence
    L.Embedding(size, outputDimensions, input_length = texts.shape[1]), #Embed the model with the number of words and size
    L.Bidirectional(L.LSTM(units, return_sequences = True)), #Make it so the model looks both forward and backward at the data
    L.GlobalMaxPool1D(), #Take the max values over time
    L.Dropout(0.3), #Make the dropout 0.3, making about a third 0 to prevent overfitting
    L.Dense(64, activation="relu"), #Create a large dense layer
    L.Dropout(0.3), #Make the dropout 0.3, making about a third 0 to prevent overfitting
    L.Dense(3) #Create a small dense layer
])


model.compile(loss = SparseCategoricalCrossentropy(from_logits = True), #Compile the model with a SparseCategorical loss function
              optimizer = 'adam', metrics = ['accuracy'] #Add an adam optimizer and collect the accuracy along the way
             )

history = model.fit(textTrain, ratingTrain, epochs = epoch, validation_split = 0.2, batch_size = batchSize) #Fit the model to the data

Epoch 1/2
344/344 [==============================] - 2314s 7s/step - loss: 0.6390 - accuracy: 0.7789 - val_loss: 0.4366 - val_accuracy: 0.8412
Epoch 2/2
344/344 [==============================] - 2290s 7s/step - loss: 0.4019 - accuracy: 0.8471 - val_loss: 0.3982 - val_accuracy: 0.8536


---

# Make Predictions

In [36]:
predict = model.predict_classes(textTest) #Predict ratings based on the model
loss, accuracy = model.evaluate(textTest, ratingTest) #Get the loss and Accuracy based on the tests

#Print the loss and accuracy
print("Test Loss: ", loss)
print("Test Accuracy: ", accuracy)

212/212 [==============================] - 380s 2s/step - loss: 0.4171 - accuracy: 0.8390
Test Loss:  0.4170674979686737
Test Accuracy:  0.8389768004417419


In [39]:
from sklearn.metrics import classification_report #Import a classification report
print(classification_report(ratingTest, predict, target_names = labels)) #Print a classification report

              precision    recall  f1-score   support

    Negative       0.73      0.76      0.74      1061
     Neutral       0.00      0.00      0.00       742
    Positive       0.86      0.98      0.92      4960

    accuracy                           0.84      6763
   macro avg       0.53      0.58      0.55      6763
weighted avg       0.75      0.84      0.79      6763



The accuracy on this came to about 84%, which is a whole lot better than my previous NLP's 76% (the Coronavirus Tweet Classification Project used spacy and not keras). This is a pretty big jump. Also, review data is a whole lot bigger than tweet data typically, yet the overall process took about the same amount of time between Keras and Spacy. 

As for the overall data, the model handled the positive values a whole lot better than the others. This is likely due to the huge desparity between the number of positive reviews versus the number of the other labels, which I did point out might come into play earlier. It did come to an 84% accuracy, though, which is a B. I will gladly take this over the 76% with Spacy, as that is an 8% difference.